In [6]:
import torchaudio
import json
import multiprocessing
from preprocessing import create_reverberated_data, create_reverberated_data_parallel, create_noisy_data, create_noisy_data_parallel
import os
from tqdm import tqdm
import torch

In [12]:
# play audio in jupyter
import IPython.display
path = "datasets/csj/concatenated/csj_eval1/A04M0051_0011923_0033367.wav"

IPython.display.Audio(path)

In [16]:
# create concat json

NAME = "csj_train_nodup_sp"
NUM_PROCS = 32
os.makedirs(f"datasets/csj/concatenated/{NAME}", exist_ok=True)
with open(f"json/{NAME}.json", "r") as f:
    data_json = json.load(f)
all_keys = list(data_json.keys())

jobs = []
queue = multiprocessing.Queue()
for i in range(NUM_PROCS):
    start = int(len(all_keys) / NUM_PROCS * i)
    end = int(len(all_keys) / NUM_PROCS * (i + 1))
    if i == NUM_PROCS - 1:
        end = len(all_keys)
    keys = all_keys[start:end]
    p = multiprocessing.Process(
        target=create_concatenated_data_parallel, args=(data_json, keys, NAME, queue)
    )
    p.start()
    jobs.append(p)

result_json = {}
for i in range(NUM_PROCS):
    result_json.update(queue.get())

for p in jobs:
    p.join()

# count num in metadata col
num = 0
for key in result_json.keys():
    num += len(result_json[key]["metainfo"]["concatenated_wav_file_paths"])

assert num == len(data_json)

with open(f"json/concat_{NAME}.json", "w") as f:
    json.dump(result_json, f, indent=4, ensure_ascii=False)

1070it [06:08,  3.68it/s]

KeyboardInterrupt: 

1085it [06:08,  4.13it/s]

1075it [06:12,  2.35it/s]

In [ ]:
# 残響の付加 (dev, eval)
NAME = "csj_train_dev"
with open(f"json/concat_{NAME}.json", "r") as f:
    data_json = json.load(f)

result_json = create_reverberated_data(data_json, NAME)
with open(f"json/reverberated_concat_{NAME}.json", "w") as f:
    json.dump(result_json, f, indent=4, ensure_ascii=False)

In [ ]:
# ノイズの付加 (dev, eval)
import multiprocessing
noise_data_json = {
    "cafeteria": {
        "wav_file_path": "/n/work3/shibutani/ASR/datasets/ku-real/noise/noise_cafeteria.wav"
    },
    "museum": {
        "wav_file_path": "/n/work3/shibutani/ASR/datasets/ku-real/noise/noise_museum.wav"
    }
}
noise_data_jsons = [noise_data_json]

NAME = "csj_train_dev"
with open(f"json/reverberated_concat_{NAME}.json", "r") as f:
    data_json = json.load(f)

all_keys = list(data_json.keys())

jobs = []
queue = multiprocessing.Queue()
NUM_PROCS = 16
for i in range(NUM_PROCS):
    start = int(len(all_keys) / NUM_PROCS * i)
    end = int(len(all_keys) / NUM_PROCS * (i + 1))
    if i == NUM_PROCS - 1:
        end = len(all_keys)
    keys = all_keys[start:end]
    p = multiprocessing.Process(
        target=create_noisy_data_parallel, args=(data_json, keys, NAME, noise_data_jsons, queue)
    )
    p.start()
    jobs.append(p)

# concat result_jsons in queue
result_json = {}
for i in range(NUM_PROCS):
    result_json.update(queue.get())

for p in jobs:
    p.join()

assert len(result_json) == len(data_json)

with open(f"json/noisy_concat_{NAME}.json", "w") as f:
    json.dump(result_json, f, indent=4, ensure_ascii=False)